# semantic-enrichment
- Valeria
- Fabian
- Cristian

In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import pandas as pd

In [2]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql

In [48]:
titulo='Forrest_Gump'
query='''
    SELECT ?name ?director ?country ?starring ?abstract
    WHERE { dbr:Forrest_Gump dbp:name ?name .
            dbr:Forrest_Gump dbo:director ?director .
            dbr:Forrest_Gump dbp:country ?country .
            dbr:Forrest_Gump dbp:starring ?starring .
            dbr:Forrest_Gump dbo:abstract ?abstract .

            FILTER (lang(?abstract) = 'en')
    }'''
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()
qres

{'head': {'link': [],
  'vars': ['name', 'director', 'country', 'starring', 'abstract']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Forrest Gump'},
    'director': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Robert_Zemeckis'},
    'country': {'type': 'literal', 'xml:lang': 'en', 'value': 'United States'},
    'starring': {'type': 'literal', 'xml:lang': 'en', 'value': 'Tom Hanks'},
    'abstract': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Forrest Gump is a 1994 American comedy-drama film directed by Robert Zemeckis and written by Eric Roth. It is based on the 1986 novel of the same name by Winston Groom and stars Tom Hanks, Robin Wright, Gary Sinise, Mykelti Williamson and Sally Field. The story depicts several decades in the life of Forrest Gump (Hanks), a slow-witted and kindhearted man from Alabama who witnesses and unwittingly influences several defining historic

In [42]:
# def find_movies_by_actor(name):
"""Function for search troght sparql films by actor
"""
query = '''
SELECT ?f 
WHERE {
?f rdf:type dbo:Film .
?f dbo:starring dbr:Tom_Hanks .

FILTER (lang(?abstract) = 'en')

SELECT ?name ?director ?country ?starring ?abstract
        WHERE { 
                
                
                dbr:'''+titulo+''' dbp:starring ?starring .
                dbr:'''+titulo+''' dbo:abstract ?abstract .
                ?f rdf:type dbo:Film .
                ?f dbo:starring dbr:Tom_Hanks .

                FILTER (lang(?abstract) = 'en')

}
'''
    
sparql.setQuery(query)

sparql.setReturnFormat(JSON)
qres = sparql.query().convert()
qres

{'head': {'link': [], 'vars': ['f']},
 'results': {'distinct': False, 'ordered': True, 'bindings': []}}

In [44]:
'''
SELECT ?name ?director ?country ?starring ?abstract
WHERE { dbr:'''+titulo+''' dbp:name ?name .
        dbr:'''+titulo+''' dbo:director ?director .
        dbr:'''+titulo+''' dbp:country ?country .
        dbr:'''+titulo+''' dbp:starring ?starring .
        dbr:'''+titulo+''' dbo:abstract ?abstract .

        FILTER (lang(?abstract) = 'en')
}''')
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()

SyntaxError: unmatched ')' (1134134871.py, line 10)

In [56]:
def detallePelicula(titulo):
    sparql.setQuery('''
        SELECT ?name ?director ?country ?starring ?abstract
        WHERE { dbr:'''+titulo+''' dbp:name ?name .
                dbr:'''+titulo+''' dbo:director ?director .
                dbr:'''+titulo+''' dbp:country ?country .
                dbr:'''+titulo+''' dbp:starring ?starring .
                dbr:'''+titulo+''' dbo:abstract ?abstract .

                FILTER (lang(?abstract) = 'en')
        }''')
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    
    diccionarios = []
    for res in qres["results"]["bindings"]:
        dic ={
            "name":res["name"]["value"],
            "director":res["director"]["value"].split("/")[-1],
            "country":res["country"]["value"],
            "abstract":res["abstract"]["value"],
            "starring":res["starring"]["value"].split("/")[-1]
        }
        diccionarios.append(dic)
    print(len(diccionarios))
    
    dics_string = json.dumps(diccionarios)  
    a_json = json.loads(dics_string)
    dataframe = pd.DataFrame.from_dict(a_json)
    dataframe.drop_duplicates()
    
    return dataframe

In [52]:
import pyvis
from pyvis.network import Network
import networkx as nx
import json

In [57]:
def grafo(df):
    G = nx.Graph()

    for a in df.itertuples():
        if a.starring != "":
            G.add_node(a.starring, size="20",labelHighlightBold=True)
            G.add_edge(a.starring, a.name)

    
    net = Network(notebook=True,directed=True)

    net.from_nx(G)
    #net.show_buttons()
    net.repulsion(node_distance=150, spring_length=100,central_gravity=0.5)
    return net.show("example.html")

In [58]:
def nuevo(titulo):
    dataframe = detallePelicula(titulo)
    dataframe.reset_index()
    dataframe.drop_duplicates()
    return dataframe

def enriquecer(dataframe,titulo):
    df=dataframe.append(detallePelicula(titulo))
    df.reset_index()
    df.drop_duplicates()
    return df

In [68]:
dataframe = nuevo("Toy_Story_3")
dataframe
grafo(dataframe)

12
